# **Análise comparativa de modelos**

In [27]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

## **1. Obtenção de dados**

Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pré-processamento.

In [5]:
df = pd.read_csv("../data/raw/wines_SPA.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,winery,Nome da vinícula,Qualitativa,Nominal
1,wine,Nome do vinho,Qualitativa,Nominal
2,year,Ano da safra do vinho,Quantitativa,Contínua
3,rating,Avaliação do vinho (1-5),Quantitativa,Discreta
4,num_reviews,Número de avaliações do vinho,Quantitativa,Discreta
5,country,País de origem do vinho,Qualitativa,Nominal
6,region,Região de origem do vinho,Qualitativa,Nominal
7,price,Preço do vinho em euros,Quantitativa,Contínua
8,type,Tipo do vinho,Qualitativa,Nominal
9,body,Classificação do corpo do vinho (1-5),Quantitativa,Discreta


## **2. Preparação de Dados**

Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.

In [9]:
target_column = 'wine'
nominal_columns = (
    df_dict
    .query("subtipo == 'Nominal' and variavel != @target_column")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'Contínua'")
    .variavel
    .to_list()
)

discreet_columns = (
     df_dict
    .query("subtipo == 'Discreta'")
    .variavel
    .to_list()
)

x = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [30]:
# tratamento de dados discrepantes
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])

continuous_preprocessor = Pipeline([
    ('missing', IterativeImputer(estimator=LinearRegression())), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

discreet_preprocessor = Pipeline([
    ('missing', IterativeImputer(estimator=LinearRegression())), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discreet', discreet_preprocessor, discreet_columns)
])

model = LogisticRegression()

## **3. Seleção de Modelos**